In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import *
from data import prepare_data_vocab, live_feat
from torch.nn.utils.rnn import unpack_sequence
from torch_geometric.data import Data


import torch.optim as optim
from torch_geometric.data import Data

In [3]:
word_to_idx, idx_to_word, data, data_emb = prepare_data_vocab("data", func=live_feat, function_num=0)

{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {}, 'temp': {}}, 1: {'static': {}, 'temp': {}}, 2: {'static': {}, 'temp': {}}, 3: {'static': {}, 'temp': {}}, 4: {'static': {}, 'temp': {}}}
{0: {'static': {'vstate': 'void *'}, 'temp': {}}, 1: {'static': {'vstate': 'void *'}, 'temp': {}}, 2: {'static': {'vstate': 'void *', 's': 'long unsigned int'}, 'temp': {}}}
{0: {'static': {'vstate': 'void *'}, 'temp': {

{'*': 0, '+': 1, '-': 2, '/': 3, 'BLANK': 4, 'M': 5, 'PHI': 6}

In [28]:
#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 64
num_layers = 7
bidirectional = True
out_dim    = 128

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [102]:
data[6][1]

[['X_47(D)->size1;', 'X_47(D)->size2;', 'work_50(D)->n;', '_1)'],
 ['work_50(D)->p;', '_2)'],
 ['19;'],
 ['y_51(D)->size;', '_3)'],
 ['19;'],
 ['c_52(D)->size;', '_4)'],
 ['19;'],
 ['0.0)'],
 ['4;'],
 ['lambda_54(D) * lambda_54(D);',
  '0.0;',
  'work_50(D)->A;',
  'gsl_matrix_submatrix (_5, 0ul, 0ul, n_48, p_49); [return slot optimization]',
  'work_50(D)->Q;',
  'gsl_matrix_submatrix (_6, 0ul, 0ul, p_49, p_49); [return slot optimization]',
  'work_50(D)->S;',
  'gsl_vector_subvector (_7, 0ul, p_49); [return slot optimization]',
  'work_50(D)->QSI;',
  'gsl_matrix_submatrix (_8, 0ul, 0ul, p_49, p_49); [return slot optimization]',
  'work_50(D)->xt;',
  'gsl_vector_subvector (_9, 0ul, p_49); [return slot optimization]',
  'work_50(D)->D;',
  'gsl_vector_subvector (_10, 0ul, p_49); [return slot optimization]',
  'work_50(D)->t;',
  'gsl_vector_subvector (_11, 0ul, n_48); [return slot optimization]'],
 ['gsl_blas_dnrm2 (&t.vector);'],
 ['__PHI (__BB11: rho_ls_56, __BB12: rho_ls_68);'],
 

In [88]:
edge_index = torch.tensor(data[6][0]).T
print(edge_index+2)

tensor([[ 2,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10, 11, 11, 12, 13,
         13, 14, 15, 16, 16, 17, 17, 18, 19, 20, 21, 21, 22, 23, 24, 25, 25, 26,
         27],
        [ 4,  3,  4,  5, 28,  6,  7, 28,  8,  9, 28, 10, 11, 28, 12, 13, 13, 14,
         20, 16, 16, 15, 17, 18, 19, 19, 27, 25, 22, 23, 24, 24, 25, 21, 26, 27,
         28]])


In [30]:
i = 2
for d in data_emb[10][1]:
    print(i, d, end= '\n')
    i += 1

2 tensor([4])
3 tensor([4])
4 tensor([3, 0, 3, 0, 2])
5 tensor([2])
6 tensor([6, 0, 1])
7 tensor([2])
8 tensor([4])
9 tensor([6])
10 tensor([6])


In [4]:
assert len(data)==len(data_emb)
for i in range(len(data)):
    assert len(data[i][2])==len(data_emb[i][1])

#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 16

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [5]:
graph = data_emb[0]
edge_index, node_embs = graph
edge_index = torch.tensor(edge_index).T

out, h, c = bb_enc(node_embs)
node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

#Step 2: gnn forward pass
graph = Data(edge_index=edge_index,
             x=node_feats)

assert graph.x.requires_grad

out = gnn(graph)
print(graph.x.shape)
print(out.shape)

torch.Size([14, 32])
torch.Size([1, 128])


In [9]:
edge_index

tensor([[ 0,  0,  1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9, 10, 10, 11,
         12],
        [ 1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9, 10, 10, 11, 12, 12,
         13]])

In [10]:
node_feats[:,0] = -1

In [11]:
node_feats

tensor([[-1.0000e+00,  2.3469e-01, -2.1152e-02,  1.1548e-01, -6.5384e-02,
          1.1938e-01, -1.0484e-01,  1.1564e-01, -2.3090e-02,  1.5918e-01,
          1.3174e-01,  1.1780e-03,  1.0086e-01, -7.8334e-02, -4.0286e-03,
          1.2905e-01, -4.2733e-02, -2.1503e-02, -1.1274e-01,  4.0716e-02,
         -1.5066e-02,  1.6134e-02, -2.5994e-02,  8.1371e-03, -3.4123e-02,
         -8.1635e-02, -5.8946e-02, -1.5579e-02,  4.9014e-02, -8.9511e-03,
         -4.2485e-03,  2.2020e-02],
        [-1.0000e+00,  1.1430e-01, -3.7418e-02,  5.8389e-02, -4.2868e-02,
          6.3044e-02, -3.6258e-02,  4.2082e-02, -2.2817e-02,  6.2591e-02,
          4.0329e-02,  6.5264e-03,  5.9489e-02, -2.8867e-02,  1.9613e-03,
          7.9815e-02, -3.9494e-02, -2.8996e-02, -1.0461e-01,  4.0669e-02,
         -8.9095e-03,  2.8706e-02, -2.9889e-02,  2.5611e-04, -3.0004e-02,
         -7.2286e-02, -4.4786e-02, -2.1987e-02,  5.3536e-02, -1.2797e-02,
         -8.0445e-03,  2.0120e-02],
        [-1.0000e+00,  2.0444e-01, -3.60

In [98]:
random_target = torch.rand((len(data_emb), 1, out_dim))

In [36]:
data_emb[0]

([[0, 1],
  [0, 2],
  [1, 2],
  [2, 3],
  [2, 4],
  [3, 4],
  [4, 5],
  [4, 6],
  [5, 6],
  [6, 7],
  [6, 8],
  [7, 8],
  [8, 9],
  [8, 10],
  [9, 10],
  [10, 11],
  [10, 12],
  [11, 12],
  [12, 13]],
 [tensor([1, 1, 1, 2, 3]),
  tensor([1]),
  tensor([6, 1, 3]),
  tensor([1]),
  tensor([6, 2, 3]),
  tensor([1]),
  tensor([6, 0, 0, 0, 2]),
  tensor([1]),
  tensor([6, 2, 3]),
  tensor([1]),
  tensor([6, 2, 3]),
  tensor([1]),
  tensor([6]),
  tensor([4])])

In [38]:
initial_node_embeddings = []
new_data_emb = []
for graph in data_emb:
    edge_index, node_embs = graph
    #if torch.tensor(edge_index).shape[0]==0:
    if len(edge_index) != 0:
        edge_index = torch.tensor(edge_index).T
    else:
        edge_index = torch.tensor([[0],[0]])
    
    out, h, c = bb_enc(node_embs)
    node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

    assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

            #Step 2: gnn forward pass
    graph = Data(edge_index=edge_index,
                        x=node_feats)
    initial_node_embeddings.append(torch.rand((graph.num_nodes, out_dim), requires_grad=True))
    new_data_emb.append([edge_index, node_embs, initial_node_embeddings])

In [39]:
from torch.utils.data import DataLoader
dataloader = DataLoader(new_data_emb, batch_size=32, shuffle=True)

In [42]:
for batch in dataloader:
    print(batch.labels)
    break

RuntimeError: stack expects each tensor to be equal size, but got [2, 4] at entry 0 and [2, 1] at entry 1

In [30]:
import random
criterion = nn.MSELoss()
optimizer = optim.Adam(gnn.parameters(), lr=0.01)

epoch_num = 300
# Training loop

for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        #if torch.tensor(edge_index).shape[0]==0:
        if len(edge_index) != 0:
            edge_index = torch.tensor(edge_index).T
        else:
            edge_index = torch.tensor([[0],[0]])
            
        
        out, h, c = bb_enc(node_embs)
        node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])
        out_act = node_feats
        assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)
        random_indices = random.sample(range(0, 128), 8)
        node_feats[:,random_indices] = -1
                #Step 2: gnn forward pass
        graph = Data(edge_index=edge_index,
                            x=node_feats)
        
        output = gnn(graph)
        
        # Unsupervised loss (dummy loss for illustration)
        #loss = criterion(output, random_target[j])
        loss = criterion(output, out_act)

        # Backward pass and optimization
        total_loss += loss.item()
        total_items += graph.x.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch%2==0 or epoch==epoch_num-1:
            avg_loss = total_loss / len(data_emb)
            print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')

        #total_loss += unsupervised_loss.item()

    # Print or log the average loss for monitoring
    if epoch%2==0 or epoch==epoch_num-1:
        print()


For 578 functions: Epoch 1/300, Average Loss: 0.04294
For 578 functions: Epoch 3/300, Average Loss: 0.04064
For 578 functions: Epoch 5/300, Average Loss: 0.03968
For 578 functions: Epoch 7/300, Average Loss: 0.03836
For 578 functions: Epoch 9/300, Average Loss: 0.03816
For 578 functions: Epoch 11/300, Average Loss: 0.03714


KeyboardInterrupt: 

In [49]:
import re

def extract_a_op_b(input_string):
    

    # Find and replace expressions within parentheses with a placeholder
    placeholder = '__PLACEHOLDER__'
    flag_par = 0
    while '(' in input_string and ')' in input_string:
        flag_par = 1
        # Find the innermost expression within parentheses
        inner_expr = re.search(r'\([^)]*\)', input_string).group(0)
        
        # Replace the inner expression with a placeholder
        input_string = input_string.replace(inner_expr, placeholder)

    # Define the original pattern
    original_pattern = r"([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])\s*([+\-*/])\s*([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])"

    # Find the first match in the modified input string
    match = re.search(original_pattern, input_string)

    if match:
        # Extract a, op, b from the match
        a = match.group(1)
        op = match.group(5)
        b = match.group(6)

        # Replace the placeholder back with the original expression
        if flag_par:
            a = a.replace(placeholder, inner_expr)
            b = b.replace(placeholder, inner_expr)

        return a, op, b
    else:
        return None

# Example usage
input_string = "_1 * 5.0e-1"
result = extract_a_op_b(input_string)

if result:
    a, op, b = result
    print("a:", a)
    print("op:", op)
    print("b:", b)
else:
    print("No match found.")


a: _1
op: *
b: 5.0e-1


In [35]:
import numpy as np
vocab = np.array(['a', 'b', 'c', 'a'])
vocab = np.unique(vocab)
word_to_idx = dict(zip(vocab, np.arange(len(vocab))))
idx_to_word = dict(zip(np.arange(len(vocab)), vocab))

In [36]:
word_to_idx

{'a': 0, 'b': 1, 'c': 2}